# Azure AI Agent Service with Bing Grounding

이 노트북은 Azure AI Agent Service와 Bing Grounding Tool을 활용하여, 사용자 질문에 대해 Bing 검색 결과를 기반으로 답변을 생성하는 예제를 다룹니다.

## 주요 내용

- Azure AI Project Client와 Bing Grounding Tool을 초기화합니다.
- Bing 연결 정보를 불러와서 Grounding Tool을 구성합니다.
- 사용자 프롬프트(질문)를 입력하면, Bing 검색을 활용한 답변을 생성하는 Agent를 동적으로 생성하고 실행합니다.
- 각 질문에 대해 에이전트의 답변과 참고한 URL 정보를 출력합니다.

이 과정을 통해, 실제 Bing 검색 결과를 기반으로 한 최신 정보 답변을 Azure AI Agent를 통해 자동화하는 방법을 실습할 수 있습니다.





In [48]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageRole, BingGroundingTool
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import AzureOpenAI
import json
load_dotenv(override=True, dotenv_path="../.env")

credential = DefaultAzureCredential()


In [43]:
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"],
    include_credentials=True
)
conn_id = bing_connection.id

print(conn_id)

bing = BingGroundingTool(connection_id=conn_id)

def populate_answer_agent_with_bing_grounding_tool(prompt: str, project_client: AIProjectClient, bing):
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
        top_p=0.9,
    )

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
        
    )

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, max_completion_tokens=1500)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    project_client.agents.delete_agent(agent.id)
    response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
        MessageRole.AGENT
    )

    return  response_message



def display_markdown_content(content: str):
    display(Markdown(content))

/subscriptions/ef750ece-3939-4a4b-ade4-cae9eabcf312/resourceGroups/rg-writer/providers/Microsoft.MachineLearningServices/workspaces/ai-project-q4u67epqs2xo4/connections/agentbingsearch


In [51]:
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]



for input in inputs:
    print(f"User input: {input}")
    rewritten_query = rewrite_query_for_search_and_llm(input, client)
    response_message = populate_answer_agent_with_bing_grounding_tool(rewritten_query['llm_query'], project_client, bing)
    if response_message:
        for text_message in response_message.text_messages:
            display_markdown_content(text_message.text.value)
        # for annotation in response_message.url_citation_annotations:
        #     print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")


User input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


삼성전자 인덕션 중 3구 이상 또는 다른 화구 수를 가진 제품을 추천드리겠습니다. 각 모델별 특징과 장점도 함께 안내해 드립니다.

---

### 1. 삼성 BESPOKE 인덕션 4구 모델
- **특징**  
  - 4구 인덕션으로 한 번에 여러 요리를 동시에 할 수 있어 가족 단위 요리에 적합합니다.  
  - BESPOKE 디자인 적용으로 주방 인테리어와 조화롭게 맞출 수 있는 다양한 색상과 마감 제공.  
  - 스마트 인덕션 기능 탑재로 안전성과 에너지 효율을 극대화.  
  - 터치 컨트롤 및 슬라이더 방식으로 사용 편리.

- **장점**  
  - 넓은 조리 공간으로 다중 요리 가능.  
  - 세련된 디자인과 맞춤형 색상으로 주방 미관 향상.  
  - 안전 센서 내장으로 과열 방지 및 어린이 보호 기능 포함.  
  - 에너지 절감 기능으로 전기료 부담 감소.

---

### 2. 삼성 3구 인덕션 빌트인 모델
- **특징**  
  - 3개의 화구로 중소규모 가정에 적합하며, 공간 효율성 뛰어남.  
  - 고화력 화구와 일반 화구가 혼합되어 다양한 용기에 맞게 사용할 수 있음.  
  - 플렉서블 존(Flexible Zone) 기능 탑재로 화구 크기 조절 가능.

- **장점**  
  - 중소형 가정에 적합한 크기와 화구 구성.  
  - 다양한 용기 대응 가능으로 요리 다양성 확대.  
  - 세련된 블랙 글라스 디자인으로 청소와 관리가 용이.  
  - 안전 잠금 기능과 자동 꺼짐 기능 제공.

---

### 3. 삼성 스마트 인덕션 5구 모델 (프리미엄 라인)
- **특징**  
  - 5개의 화구를 갖추어 대가족이나 대용량 요리 시 매우 편리.  
  - 스마트 연동 기능으로 IoT 기반 원격 제어 가능.  
  - 고화력과 저화력 화구가 적절히 배치되어 요리 효율 극대화.

- **장점**  
  - 대용량 요리에 최적화된 화구 구성.  
  - 원격으로 인덕션 상태 확인 및 조작 가능해 편리함 증대.  
  - 내구성 높은 세라믹 글라스 상판 사용.  
  - 다양한 안전 기능 내장으로 사고 예방.

---

각 인덕션 모델은 설치 공간과 용도, 요리 스타일에 따라 선택할 수 있으니 참고하시기 바랍니다. 추가로 원하시는 기능이나 디자인 요구사항이 있다면 알려주시면 더욱 맞춤형 추천도 가능합니다!

User input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


부모님께 선물하기 좋은 삼성전자 TV를 추천드릴 때, 사용 용도와 가격대를 고려해 몇 가지 모델을 소개해 드릴게요. 각 모델의 특징과 장점도 함께 정리했습니다.

---

## 1. 기본 활용 및 가성비 중심 추천: 삼성 QN90B (Neo QLED)

- **화질**: Neo QLED 기술로 밝기와 명암비가 뛰어나고, 색 표현력이 우수해 TV 시청과 영화 감상에 최적화
- **사이즈**: 50~85인치 다양한 크기 옵션 제공
- **스마트 기능**: Tizen OS 탑재, 다양한 스트리밍 앱 지원, 스마트 홈 연동 가능
- **장점**: 프리미엄 화질에 비해 가격이 상대적으로 합리적, 반사광 제어 우수해 밝은 거실에 적합

---

## 2. 영화 감상과 넷플릭스 등 OTT 시청 중심: 삼성 The Frame 2022/2023

- **화질**: QLED 패널로 선명한 화질 제공
- **디자인**: 액자처럼 벽에 걸어 인테리어 효과 극대화, 아트 모드로 유명 작품이나 가족사진을 전시 가능
- **사이즈**: 43~75인치
- **스마트 기능**: 스마트 TV 기능 탑재
- **장점**: TV가 꺼져 있을 때도 인테리어 소품으로 활용 가능, 디자인 중시하는 분들께 추천

---

## 3. 프리미엄 초대형 TV: 삼성 Neo QLED QN900B (8K)

- **화질**: 8K 해상도와 Neo QLED로 최고급 화질 경험 가능
- **사이즈**: 65~85인치
- **기능**: 고사양 게임용 HDMI 2.1 지원, 고성능 사운드 시스템 내장
- **장점**: 뛰어난 화질과 대형 화면으로 홈시네마 구축 가능, 최신 기술 집약

---

## 4. 가성비 스마트 TV: 삼성 AU8000

- **화질**: Crystal UHD 패널로 적당한 화질, 일상적인 TV 시청에 적합
- **사이즈**: 43~75인치
- **스마트 기능**: 기본 스마트 TV 기능 탑재
- **장점**: 부담 없는 가격으로 스마트 TV 경험 가능, 부모님 사용에 무리 없는 성능

---

### 요약

| 모델명           | 주요특징                          | 추천용도               | 가격대          |
|------------------|----------------------------------|------------------------|-----------------|
| QN90B            | Neo QLED, 밝고 선명한 화질       | 영화, 드라마 감상       | 중고가          |
| The Frame        | 액자형 디자인, 아트 모드          | 인테리어+TV 활용       | 중고가          |
| QN900B (8K)      | 초고해상도, 프리미엄 스펙         | 홈시네마, 고급형       | 고가            |
| AU8000           | 가성비 좋은 Crystal UHD          | 기본 TV+스마트 기능    | 저중가          |

---

부모님께서 주로 어떤 용도로 TV를 사용하실지(예: 드라마/영화 감상, 뉴스/예능 시청, 인테리어 중시 등)를 알려주시면 좀 더 맞춤형으로 추천해 드릴 수 있습니다! 필요하시면 사이즈나 가격대도 알려 주세요.

User input: 삼성전자 25년 제품이 작년 대비 좋아진것은


...

삼성전자에서 판매하는 JBL과 하만카돈(Harman Kardon) 스피커는 모두 하만(Harman) 그룹 소속 브랜드로서, 고품질 사운드를 제공하지만, 각각의 브랜드가 지향하는 음향 특성과 디자인, 기능에서 차별점이 있습니다. 아래에 주요 차이점을 음질, 디자인, 가격, 기능 측면에서 비교해 자세히 설명드리겠습니다.

---

## 1. 음질

- **JBL**
  - 베이스가 강하고 파워풀한 사운드를 강조하는 브랜드
  - 클럽이나 파티, 야외에서 강력한 저음과 큰 음량을 필요로 하는 환경에 적합
  - 고음과 중음도 명확하지만, 베이스가 강조되어 음악 감상에 다이내믹함을 더함

- **하만카돈**
  - 더 균형 잡히고 세련된 사운드를 제공
  - 중음과 고음의 해상도가 높고 자연스러운 음질로 클래식, 재즈, 보컬 음악에 적합
  - 공간감과 음장의 넓이를 중요시하는 음향 설계

---

## 2. 디자인

- **JBL**
  - 주로 강인하고 튼튼한 디자인, 스포츠나 야외 활동에 적합한 내구성 강조
  - 다채로운 컬러와 현대적인 스타일, 휴대성을 고려한 컴팩트한 크기 디자인
  - 방수, 방진 기능이 잘 갖춰진 모델이 많음

- **하만카돈**
  - 고급스럽고 미니멀한 프리미엄 디자인
  - 메탈과 패브릭 등 고급 소재를 사용해 인테리어 소품으로도 손색없음
  - 감성적이고 세련된 외관, 주로 실내 사용에 적합한 디자인

---

## 3. 가격

- **JBL**
  - 대체로 합리적인 가격대의 모델이 많음
  - 중저가부터 중고가까지 폭넓은 라인업으로 접근성 높음

- **하만카돈**
  - 프리미엄 라인에 속하는 만큼 가격대가 상대적으로 높음
  - 고품질 음향과 디자인에 투자하는 사용자 대상

---

## 4. 기능

- **JBL**
  - 블루투스 연결과 휴대성에 중점
  - 일부 모델은 파티 부스트(PartyBoost) 기능으로 여러 대 스피커 연결 가능
  - IPX 등급의 방수 기능 탑재 모델 다수

- **하만카돈**
  - 고해상도 오디오 지원과 무선 스트리밍 기능 강화
  - 스마트홈 연동, 음성 인식 지원 모델도 있음
  - 일부 모델은 멀티룸 시스템과 호환 가능

---

### 요약

| 항목     | JBL                              | 하만카돈                          |
|----------|---------------------------------|----------------------------------|
| 음질     | 강력한 베이스, 다이내믹한 사운드 | 균형 잡힌 고해상도, 자연스러운 음질 |
| 디자인   | 튼튼하고 컬러풀한 스포츠형       | 고급스럽고 미니멀한 프리미엄 디자인 |
| 가격     | 합리적, 중저가부터 중고가까지    | 프리미엄 가격대                    |
| 기능     | 휴대성, 방수, 파티 부스트        | 고해상도, 스마트 기능, 멀티룸 지원 |

---

필요하시면 삼성전자 공식 홈페이지나 제품 상세 페이지에서 모델별 스펙과 기능을 확인해 드릴 수 있습니다. 특정 모델 비교도 가능합니다.

User input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요


갤럭시 버즈 이어버드 한쪽을 새로 구매해 기존 이어버드와 정상적으로 페어링하는 방법을 단계별로 안내해드리겠습니다.

---

## 갤럭시 버즈 이어버드 한쪽 새로 구매 시 기존 이어버드와 페어링 방법

### 1. 양쪽 이어버드 케이스에 넣고 완전 충전하기
- 새로 산 이어버드와 기존 이어버드를 모두 충전 케이스에 넣고, 케이스에 충전 상태가 충분히 될 때까지 충전합니다.

### 2. 기존에 연결된 기기에서 갤럭시 버즈 연결 해제하기
- 스마트폰에서 '설정 > 연결 > 블루투스' 메뉴에 들어가 기존 갤럭시 버즈 연결을 해제합니다.
- 또는 블루투스 목록에서 갤럭시 버즈를 선택해 '연결 해제' 또는 '기기 제거'를 합니다.

### 3. 갤럭시 버즈 케이스 열기
- 충전 케이스 뚜껑을 열고, 새 이어버드와 기존 이어버드가 모두 케이스 안에 제대로 장착되어 있는지 확인하세요.

### 4. 리셋(초기화) 하기
- 이어버드를 케이스에 넣은 상태에서 이어버드 터치 센서를 약 10초 이상 눌러 초기화를 진행합니다.
- 또는 갤럭시 웨어러블 앱에서 ‘이어버드 재설정’ 메뉴를 찾아 초기화를 진행할 수 있습니다.

### 5. 갤럭시 웨어러블 앱에서 새 이어버드 연결하기
- 스마트폰에서 ‘갤럭시 웨어러블’ 앱을 실행합니다.
- 기존 이어버드가 등록되어 있다면, 앱에서 ‘이어버드 추가’ 또는 ‘새 이어버드 연결’ 옵션을 선택합니다.
- 앱의 안내에 따라 새 이어버드와 기존 이어버드가 함께 페어링 될 때까지 기다립니다.

### 6. 양쪽 이어버드 정상 작동 확인하기
- 페어링이 완료되면 양쪽 이어버드에서 소리가 정상적으로 나오는지 확인합니다.
- 만약 한쪽 이어버드만 작동한다면, 초기화 과정을 다시 진행하거나 배터리 상태를 확인하세요.

---

### 참고사항
- 이어버드 모델에 따라 초기화 방법이 조금씩 다를 수 있습니다. 갤럭시 웨어러블 앱에서 지원하는 모델에 맞게 안내를 받으실 수 있습니다.
- 새로 산 이어버드가 기존 이어버드와 동일한 모델인지 꼭 확인해주세요. 모델이 다르면 페어링이 어려울 수 있습니다.

필요시 모델명 알려주시면 좀 더 상세한 맞춤 안내 도와드리겠습니다!

User input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나


삼성전자 갤럭시 S25와 S24 모델 간 무게 차이에 대해 조사한 결과, 갤럭시 S24 기본 모델의 무게는 약 168g입니다. 반면, 갤럭시 S25 엣지 모델은 163g으로 알려져 있습니다. 

따라서, 두 모델 간 무게 차이는 약 5g 정도로 갤럭시 S25 엣지가 더 가볍습니다.

- 갤럭시 S24 무게: 약 168g
- 갤럭시 S25 엣지 무게: 약 163g

이로 인해 S25가 좀 더 초슬림, 초경량 설계가 적용된 모델임을 알 수 있습니다.【13:1†source】【13:4†source】